### **TODO and Notes**

 - [x] Make dataframes
 - [x] Convert datetimes
 - [x] rename date columns
 - [x] Find Nans
 - [x] Re-freq and fill blanks
 - [x] Turn logs into daily totals
 - [x] Save backup CSVs
 - [x] Combine dataframe
 
 **Notes**
Sample for iterating through different offsets 
```python
df["Input"].corr(df["Output"].shift(-1), method = 'pearson', min_periods = 1) #1
```
and more iteration 
```python
 xcov_monthly = [crosscorr(datax, datay, lag=i) for i in range(12)]
```
from [here](https://stackoverflow.com/questions/33171413/cross-correlation-time-lag-correlation-with-pandas)


## Imports, data, checks

In [1]:
import numpy as np
import requests
import pandas as pd
import datetime
from urllib.request import urlopen
import json
from bokeh.models import CategoricalColorMapper, NumeralTickFormatter, HoverTool
from bokeh.models import ColumnDataSource, Grid, LinearAxis, Plot, VBar
from bokeh.plotting import output_notebook, figure
from bokeh.io import reset_output, show, output_file
from bokeh.layouts import column, row

The vaccine, cases, and deaths source data were relatively easy to grab diretly from the [Larimer county dashboard](https://www.larimer.org/health/communicable-disease/coronavirus-covid-19/larimer-county-positive-covid-19-numbers#/app?tab=risk) as the CSVs download through urls.

In [2]:
larimer_vac_source = pd.read_csv('https://speedtest.larimer.org/covid/index.php?file=vaccinations&csv')

larimer_cases_source = pd.read_csv('https://speedtest.larimer.org/covid/cases.csv', parse_dates=['ReportedDate'])

larimer_deaths_source = pd.read_csv('https://larimer-county-data-lake.s3-us-west-2.amazonaws.com/Public/covid/covid_deaths.csv?t=1631890252549')


The hospitalization data was much more tricky (at least finding a simple solution was tricky) I spent several hours in webscraping research and attempts purgatory. I checked BeautifulSoup, html5lib, lxml, etc. in multiple combinations and none of them had straightforward solutions because the table for hospitalizations is actually rendered through javascript so there is nothing to scrape without actually clicking the buttons. I started down the Selenium and phantomjs path but it seemed like a nightmare. I found this lifesaving article at [Towards Data Science](https://towardsdatascience.com/data-science-skills-web-scraping-javascript-using-python-97a29738353f) which shows how to find specific XHR request urls in the browser developer tools. The requested URL for the rendered table is a pretty vanilla json and not behind any authorization so there is a pretty clean way to get to it. Praise Satan I didn't have to use Selenium.  

In [3]:
url = 'https://larimer-county-data-lake.s3-us-west-2.amazonaws.com/Public/covid/covid_patient_trend.json?t=1632506827395'

response = urlopen(url)
json_data = response.read().decode('utf-8', 'replace')

d = json.loads(json_data)
larimer_hosp_source = pd.json_normalize(d['data'])

In [4]:
# make .csv backups of source data

larimer_vac_source.to_csv('larimer_vac_backup.csv')

larimer_cases_source.to_csv('larimer_cases_backup.csv')

larimer_deaths_source.to_csv('larimer_deaths_backup.csv')

larimer_hosp_source.to_csv('larimer_hosp_backup.csv')

Re-read the backup CSVs so that the notebook runs locally from this point forward.

In [5]:
larimer_vac = pd.read_csv('larimer_vac_backup.csv')

larimer_cases = pd.read_csv('larimer_cases_backup.csv')

larimer_deaths = pd.read_csv('larimer_deaths_backup.csv')

larimer_hosp = pd.read_csv('larimer_hosp_backup.csv')

So now we have all of our dataframes

In [6]:
display(larimer_vac)

display(larimer_cases)

display(larimer_deaths)

display(larimer_hosp)

,Unnamed: 0,Date,daily number of doses received by Larimer County residents,total number of doses recevied by residents,daily number of residents receiving first dose,total number of residents receiving first dose,daily number of residents vaccinated,total number of residents vaccinated,daily number of 70+ vaccinated,total number of 70+ vaccinated,...,daily number of Latinx residents vaccinated,total of Latinx residents vaccinated,daily number of White non-Latinx residents vaccinated,total of White non-Latinx residents vaccinated,daily number of non-White non-Latinx residents vaccinated,total of non-White non-Latinx residents vaccinated,dailyUnknown,totalUnknown,daily_additional_doses,total_additional_doses
0,0,12/14/2020,32,32,32,32,1,1,0.0,0,...,0.0,0,1,1,0.0,0,NaN,0,0,0
1,1,12/15/2020,14,46,14,46,0,1,NaN,0,...,NaN,0,0,1,0.0,0,NaN,0,0,0
2,2,12/16/2020,304,350,304,350,0,1,0.0,0,...,0.0,0,0,1,0.0,0,0.0,0,0,0
3,3,12/17/2020,989,1339,989,1339,0,1,0.0,0,...,0.0,0,0,1,0.0,0,0.0,0,0,0
4,4,12/18/2020,1053,2392,1053,2392,2,3,0.0,0,...,0.0,0,2,3,0.0,0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,402,1/20/2022,763,599815,145,253989,146,233276,0.0,34815,...,17.0,13564,110,197017,10.0,14038,9.0,8657,479,129802
403,403,1/21/2022,1149,600964,166,254155,170,233446,7.0,34822,...,14.0,13578,132,197149,11.0,14049,13.0,8670,824,130626
404,404,1/22/2022,777,601741,88,254243,102,233548,0.0,34822,...,14.0,13592,73,197222,6.0,14055,9.0,8679,590,131216
405,405,1/23/2022,172,601913,22,254265,41,233589,1.0,34823,...,5.0,13597,30,197252,6.0,14061,0.0,8679,110,131326


,Unnamed: 0,CaseCount,ReportedDate,Sex,Age,Type,City
0,0,1,2020-03-09,Female,52.0,Confirmed,Johnstown
1,1,2,2020-03-15,Male,49.0,Confirmed,Fort Collins
2,2,3,2020-03-17,Female,53.0,Confirmed,Fort Collins
3,3,4,2020-03-17,Female,94.0,Confirmed,Loveland
4,4,5,2020-03-18,Male,49.0,Confirmed,Fort Collins
...,...,...,...,...,...,...,...
68234,68234,70166,2022-01-25,Male,77.0,Confirmed,Fort Collins
68235,68235,70167,2022-01-25,Female,79.0,Probable,Fort Collins
68236,68236,70168,2022-01-25,Male,79.0,Confirmed,Fort Collins
68237,68237,70169,2022-01-25,Female,79.0,Confirmed,Fort Collins


,Unnamed: 0,death_id,death_date,age,gender,city,case_status,count
0,0,a0U5w00000edbfjEAA,2020-03-09,91,Female,Loveland,Probable,1
1,1,a0U5w00000edbfiEAA,2020-03-13,95,Female,Loveland,Probable,2
2,2,a0U5w00000edbfOEAQ,2020-03-15,90,Female,Loveland,Probable,3
3,3,a0U5w00000edbfMEAQ,2020-03-25,74,Female,Loveland,Confirmed,4
4,4,a0U5w00000edbfJEAQ,2020-03-25,87,Female,Fort Collins,Confirmed,5
...,...,...,...,...,...,...,...,...
424,424,a0U5w00000fp3jpEAA,2022-01-09,62,Female,Loveland,Confirmed,425
425,425,a0U5w00000fp3juEAA,2022-01-09,45,Male,Loveland,Confirmed,426
426,426,a0U5w00000fp4VVEAY,2022-01-11,85,Female,Fort Collins,Confirmed,427
427,427,a0U5w00000fp55DEAQ,2022-01-13,38,Female,Loveland,Confirmed,428


,Unnamed: 0,Date,admission_count,kpi_admits_indicator,inpatient_count,kpi_patient_indicator,inpatient_count_pct_change
0,0,2020-03-31T00:00:00.000Z,NaN,NaN,47,0,NaN
1,1,2020-04-01T00:00:00.000Z,NaN,NaN,46,0,NaN
2,2,2020-04-02T00:00:00.000Z,NaN,NaN,46,0,NaN
3,3,2020-04-03T00:00:00.000Z,2.0,0.0,46,0,NaN
4,4,2020-04-04T00:00:00.000Z,1.0,0.0,42,0,NaN
...,...,...,...,...,...,...,...
451,451,2022-01-18T00:00:00.000Z,23.0,1.0,90,1,32.352941
452,452,2022-01-19T00:00:00.000Z,22.0,1.0,104,1,20.930233
453,453,2022-01-20T00:00:00.000Z,18.0,1.0,101,1,20.238095
454,454,2022-01-24T00:00:00.000Z,11.0,0.0,106,1,34.177215


Make .CSVs of the raw data as a backup, and then re-read the DFs from those so that this notebook continues to work from that point forward even if the data sources are disconnected.

This looks like pretty good start. We'll have to make all the datetimes match and the **hospitalization** and **vaccine** data are daily totals while the **death** and **case counts** data is a case log (a row for each case) so we'll have to do some grouping to get that to match, that will come later.

## Explore, clean, manipulate

In [7]:
dfs = [larimer_vac, larimer_deaths, larimer_cases, larimer_hosp]

def get_obj_col():
    for df in dfs:
        print(list(df.select_dtypes(['object']).columns))

get_obj_col()

['Date']
['death_id', 'death_date', 'gender', 'city', 'case_status']
['ReportedDate', 'Sex', 'Type', 'City']
['Date']


---
I did this and don't like it
```python

dfs = [larimer_vac, larimer_deaths, larimer_cases, larimer_hosp]
df_names = ['larimer_vac', 'larimer_deaths', 'larimer_cases', 'larimer_hosp']


def get_obj_col():
    for df in dfs:
        obj_cols.append(list(df.select_dtypes(['object']).columns))
    zip(df_names, dfs)
    
obj_cols = []
get_obj_col()
zipped_list = zip(df_names, obj_cols)
print(tuple(zipped_list)
```
---

In [8]:
print(larimer_cases.dtypes)
print(larimer_hosp.dtypes)

Unnamed: 0        int64
CaseCount         int64
ReportedDate     object
Sex              object
Age             float64
Type             object
City             object
dtype: object
Unnamed: 0                      int64
Date                           object
admission_count               float64
kpi_admits_indicator          float64
inpatient_count                 int64
kpi_patient_indicator           int64
inpatient_count_pct_change    float64
dtype: object


Convert date columns from each df to datetimes

In [9]:
larimer_vac['Date'] = pd.to_datetime(larimer_vac['Date']).dt.tz_localize(None)
larimer_deaths['Date'] = pd.to_datetime(larimer_deaths['death_date']).dt.tz_localize(None)
larimer_cases['Date'] = pd.to_datetime(larimer_cases['ReportedDate']).dt.tz_localize(None)
larimer_hosp['Date'] = pd.to_datetime(larimer_hosp['Date']).dt.tz_localize(None)

```pd.to_datetime``` was sufficient for most of the dfs but the hospital data was TZ aware and I wanted all of them to match so had to add the ```.dt.tz_localize(None)``` 

In [10]:
def check_date_type():
    for df in dfs:
        print(list(df.select_dtypes(['datetime64']).columns))

check_date_type()

['Date']
['Date']
['Date']
['Date']


In [11]:
# create daily cases from case log
daily_cases = larimer_cases.groupby(['Date']).count().reset_index()

display(daily_cases)
display(daily_cases.dtypes)
print(f"Total case check {daily_cases['CaseCount'].sum()}")
display(daily_cases.describe()) 

,Date,Unnamed: 0,CaseCount,ReportedDate,Sex,Age,Type,City
0,2020-03-09,1,1,1,1,1,1,1
1,2020-03-15,1,1,1,1,1,1,1
2,2020-03-17,2,2,2,2,2,2,2
3,2020-03-18,1,1,1,1,1,1,1
4,2020-03-19,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...
676,2022-01-21,593,593,593,593,593,593,593
677,2022-01-22,494,494,494,494,494,494,494
678,2022-01-23,450,450,450,450,450,450,450
679,2022-01-24,444,444,444,444,444,444,444


Date            datetime64[ns]
Unnamed: 0               int64
CaseCount                int64
ReportedDate             int64
Sex                      int64
Age                      int64
Type                     int64
City                     int64
dtype: object

Total case check 68239


,Unnamed: 0,CaseCount,ReportedDate,Sex,Age,Type,City
count,681.000000,681.000000,681.000000,681.000000,681.000000,681.000000,681.000000
mean,100.204112,100.204112,100.204112,100.204112,100.048458,100.204112,100.204112
std,139.018898,139.018898,139.018898,139.018898,138.885339,139.018898,139.018898
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000
50%,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000
75%,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000
max,1099.000000,1099.000000,1099.000000,1099.000000,1096.000000,1099.000000,1099.000000


In [12]:
# create daily deaths from death log
daily_deaths = larimer_deaths.groupby(['Date']).count().reset_index()

display(daily_deaths)
display(daily_deaths.dtypes)
print(f"Total death check {daily_deaths['count'].sum()}")
display(daily_deaths.describe()) 

,Date,Unnamed: 0,death_id,death_date,age,gender,city,case_status,count
0,2020-03-09,1,1,1,1,1,1,1,1
1,2020-03-13,1,1,1,1,1,1,1,1
2,2020-03-15,1,1,1,1,1,1,1,1
3,2020-03-25,2,2,2,2,2,2,2,2
4,2020-03-29,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...
257,2022-01-06,2,2,2,2,2,2,2,2
258,2022-01-09,2,2,2,2,2,2,2,2
259,2022-01-11,1,1,1,1,1,1,1,1
260,2022-01-13,1,1,1,1,1,1,1,1


Date           datetime64[ns]
Unnamed: 0              int64
death_id                int64
death_date              int64
age                     int64
gender                  int64
city                    int64
case_status             int64
count                   int64
dtype: object

Total death check 429


,Unnamed: 0,death_id,death_date,age,gender,city,case_status,count
count,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000
mean,1.637405,1.637405,1.637405,1.637405,1.637405,1.637405,1.637405,1.637405
std,1.062336,1.062336,1.062336,1.062336,1.062336,1.062336,1.062336,1.062336
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
max,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000


In [13]:
daily_cases.set_index('Date', inplace=True)

daily_deaths.set_index('Date', inplace=True)

larimer_vac.set_index('Date', inplace=True)

larimer_hosp.set_index('Date', inplace=True)

In [14]:
# daily_cases.index = pd.to_datetime(daily_cases.index)
# daily_cases = daily_cases.resample("1D").mean()
# daily_cases


**Try this**

```python
x.dt = pd.to_datetime(x.dt)
```
One-liner using mostly @ayhan's ideas while incorporating stack/unstack and fill_value

```python
x.set_index(
    ['dt', 'user']
).unstack(
    fill_value=0
).asfreq(
    'D', fill_value=0
).stack().sort_index(level=1).reset_index()
```
**or this might be better**
```python
s.asfreq('D'))
```


In [15]:
larimer_hosp['admission_count'] = larimer_hosp['admission_count'].astype("Int64")
larimer_hosp

,Unnamed: 0,admission_count,kpi_admits_indicator,inpatient_count,kpi_patient_indicator,inpatient_count_pct_change
Date,,,,,,
2020-03-31,0,<NA>,NaN,47,0,NaN
2020-04-01,1,<NA>,NaN,46,0,NaN
2020-04-02,2,<NA>,NaN,46,0,NaN
2020-04-03,3,2,0.0,46,0,NaN
2020-04-04,4,1,0.0,42,0,NaN
...,...,...,...,...,...,...
2022-01-18,451,23,1.0,90,1,32.352941
2022-01-19,452,22,1.0,104,1,20.930233
2022-01-20,453,18,1.0,101,1,20.238095


In [16]:
larimer_hosp[larimer_hosp.index.duplicated()]

,Unnamed: 0,admission_count,kpi_admits_indicator,inpatient_count,kpi_patient_indicator,inpatient_count_pct_change
Date,,,,,,
2021-12-15,428,6,0.0,81,1,5.194805
2021-12-15,429,6,0.0,81,1,8.000000
2021-12-15,430,6,0.0,81,1,5.194805


This weird 'Unamed:0" column appeared when I switched to using dfs from the backup CSVs so I had to drop it in place to make the following duplicate drops work.


In [17]:
larimer_hosp.drop(['Unnamed: 0'], axis = 1, inplace=True)

In [18]:
larimer_hosp.drop_duplicates(keep=False,inplace = True)

In [19]:
larimer_hosp[larimer_hosp.index.duplicated()]

,admission_count,kpi_admits_indicator,inpatient_count,kpi_patient_indicator,inpatient_count_pct_change
Date,,,,,


In [20]:
daily_cases_filled = daily_cases.asfreq('D',fill_value=0)
daily_deaths_filled = daily_deaths.asfreq('D',fill_value=0)
larimer_vac_filled = larimer_vac.asfreq('D',fill_value=0)
larimer_hosp_filled = larimer_hosp.asfreq('D',fill_value=0)



## Quantify missing data

In [21]:
print(daily_cases_filled.isna().sum().sum())
print(daily_deaths_filled .isna().sum().sum())
print(larimer_vac_filled .isna().sum().sum())
print(larimer_hosp_filled.isna().sum().sum())


0
0
20
38


In [22]:
larimer_hosp_filled = larimer_hosp_filled.fillna(0)
larimer_vac_filled = larimer_vac_filled.fillna(0)

In [23]:
print(daily_cases_filled.isna().sum().sum())
print(daily_deaths_filled .isna().sum().sum())
print(larimer_vac_filled .isna().sum().sum())
print(larimer_hosp_filled.isna().sum().sum())


0
0
0
0


In [24]:
display(daily_cases_filled)
display(daily_deaths_filled)
display(larimer_vac_filled)
display(larimer_hosp_filled)

,Unnamed: 0,CaseCount,ReportedDate,Sex,Age,Type,City
Date,,,,,,,
2020-03-09,1,1,1,1,1,1,1
2020-03-10,0,0,0,0,0,0,0
2020-03-11,0,0,0,0,0,0,0
2020-03-12,0,0,0,0,0,0,0
2020-03-13,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...
2022-01-21,593,593,593,593,593,593,593
2022-01-22,494,494,494,494,494,494,494
2022-01-23,450,450,450,450,450,450,450


,Unnamed: 0,death_id,death_date,age,gender,city,case_status,count
Date,,,,,,,,
2020-03-09,1,1,1,1,1,1,1,1
2020-03-10,0,0,0,0,0,0,0,0
2020-03-11,0,0,0,0,0,0,0,0
2020-03-12,0,0,0,0,0,0,0,0
2020-03-13,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...
2022-01-12,0,0,0,0,0,0,0,0
2022-01-13,1,1,1,1,1,1,1,1
2022-01-14,0,0,0,0,0,0,0,0


,Unnamed: 0,daily number of doses received by Larimer County residents,total number of doses recevied by residents,daily number of residents receiving first dose,total number of residents receiving first dose,daily number of residents vaccinated,total number of residents vaccinated,daily number of 70+ vaccinated,total number of 70+ vaccinated,daily number of 70+ at least one dose,...,daily number of Latinx residents vaccinated,total of Latinx residents vaccinated,daily number of White non-Latinx residents vaccinated,total of White non-Latinx residents vaccinated,daily number of non-White non-Latinx residents vaccinated,total of non-White non-Latinx residents vaccinated,dailyUnknown,totalUnknown,daily_additional_doses,total_additional_doses
Date,,,,,,,,,,,,,,,,,,,,,
2020-12-14,0,32,32,32,32,1,1,0.0,0,1.0,...,0.0,0,1,1,0.0,0,0.0,0,0,0
2020-12-15,1,14,46,14,46,0,1,0.0,0,0.0,...,0.0,0,0,1,0.0,0,0.0,0,0,0
2020-12-16,2,304,350,304,350,0,1,0.0,0,2.0,...,0.0,0,0,1,0.0,0,0.0,0,0,0
2020-12-17,3,989,1339,989,1339,0,1,0.0,0,11.0,...,0.0,0,0,1,0.0,0,0.0,0,0,0
2020-12-18,4,1053,2392,1053,2392,2,3,0.0,0,15.0,...,0.0,0,2,3,0.0,0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-20,402,763,599815,145,253989,146,233276,0.0,34815,10.0,...,17.0,13564,110,197017,10.0,14038,9.0,8657,479,129802
2022-01-21,403,1149,600964,166,254155,170,233446,7.0,34822,2.0,...,14.0,13578,132,197149,11.0,14049,13.0,8670,824,130626
2022-01-22,404,777,601741,88,254243,102,233548,0.0,34822,1.0,...,14.0,13592,73,197222,6.0,14055,9.0,8679,590,131216


,admission_count,kpi_admits_indicator,inpatient_count,kpi_patient_indicator,inpatient_count_pct_change
Date,,,,,
2020-03-31,0,0.0,47,0,0.000000
2020-04-01,0,0.0,0,0,0.000000
2020-04-02,0,0.0,0,0,0.000000
2020-04-03,2,0.0,46,0,0.000000
2020-04-04,1,0.0,42,0,0.000000
...,...,...,...,...,...
2022-01-21,0,0.0,0,0,0.000000
2022-01-22,0,0.0,0,0,0.000000
2022-01-23,0,0.0,0,0,0.000000


In [25]:
display(len(larimer_vac_filled))
display(len(larimer_hosp_filled))
display(len(daily_cases_filled))
display(len(daily_deaths_filled))


407

666

688

679

In [26]:
# valid_entries = larimer_vac.count()
# total_rows = len(larimer_vac.index)
# missing_data = total_rows - valid_entries
# missing_data

```python
merge_ordered(df1,
              df2,
              fill_method="ffill",
              on='column',
              how='outer'
```

- [x] Experimenting with merging on 'Date' column but it's been put back as an int instead of a datetime so may need to re-type that in all the DFs
- [x] Need to rename the date column in one of the frames so they can all be merged

In [27]:
# daily_cases_filled['Date'] = pd.to_datetime(daily_cases_filled['Date']).dt.tz_localize(None)
# daily_deaths_filled['Date'] = pd.to_datetime(daily_deaths_filled['Date']).dt.tz_localize(None)
# larimer_hosp_filled['Date'] = pd.to_datetime(larimer_hosp_filled['Date']).dt.tz_localize(None)
# larimer_vac_filled['Date'] = pd.to_datetime(larimer_vac_filled['Date']).dt.tz_localize(None)

In [28]:
death_case = pd.merge_ordered(
    daily_deaths_filled,
    daily_cases_filled,
    fill_method=None,
    on='Date',
    how='outer')

death_case

,Date,Unnamed: 0_x,death_id,death_date,age,gender,city,case_status,count,Unnamed: 0_y,CaseCount,ReportedDate,Sex,Age,Type,City
0,2020-03-09,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1,1,1,1,1,1
1,2020-03-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0
2,2020-03-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0
3,2020-03-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0
4,2020-03-13,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,2022-01-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,593,593,593,593,593,593,593
684,2022-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,494,494,494,494,494,494,494
685,2022-01-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,450,450,450,450,450,450,450
686,2022-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,444,444,444,444,444,444,444


In [29]:
death_case_hosp = pd.merge_ordered(
    death_case,
    larimer_hosp_filled,
    fill_method=None,
    on='Date',
    how='outer')

death_case_hosp

,Date,Unnamed: 0_x,death_id,death_date,age,gender,city,case_status,count,Unnamed: 0_y,...,ReportedDate,Sex,Age,Type,City,admission_count,kpi_admits_indicator,inpatient_count,kpi_patient_indicator,inpatient_count_pct_change
0,2020-03-09,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,...,1,1,1,1,1,<NA>,NaN,NaN,NaN,NaN
1,2020-03-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,<NA>,NaN,NaN,NaN,NaN
2,2020-03-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,<NA>,NaN,NaN,NaN,NaN
3,2020-03-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,<NA>,NaN,NaN,NaN,NaN
4,2020-03-13,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,...,0,0,0,0,0,<NA>,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,2022-01-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,593,...,593,593,593,593,593,0,0.0,0.0,0.0,0.000000
684,2022-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,494,...,494,494,494,494,494,0,0.0,0.0,0.0,0.000000
685,2022-01-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,450,...,450,450,450,450,450,0,0.0,0.0,0.0,0.000000
686,2022-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,444,...,444,444,444,444,444,11,0.0,106.0,1.0,34.177215


In [30]:
combo_df = pd.merge_ordered(
    death_case_hosp,
    larimer_vac_filled,
    fill_method=None,
    on='Date',
    how='outer')

combo_df

,Date,Unnamed: 0_x,death_id,death_date,age,gender,city,case_status,count,Unnamed: 0_y,...,daily number of Latinx residents vaccinated,total of Latinx residents vaccinated,daily number of White non-Latinx residents vaccinated,total of White non-Latinx residents vaccinated,daily number of non-White non-Latinx residents vaccinated,total of non-White non-Latinx residents vaccinated,dailyUnknown,totalUnknown,daily_additional_doses,total_additional_doses
0,2020-03-09,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-03-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-03-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-03-13,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,2022-01-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,593,...,14.0,13578.0,132.0,197149.0,11.0,14049.0,13.0,8670.0,824.0,130626.0
684,2022-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,494,...,14.0,13592.0,73.0,197222.0,6.0,14055.0,9.0,8679.0,590.0,131216.0
685,2022-01-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,450,...,5.0,13597.0,30.0,197252.0,6.0,14061.0,0.0,8679.0,110.0,131326.0
686,2022-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,444,...,13.0,13610.0,46.0,197298.0,5.0,14066.0,2.0,8681.0,151.0,131477.0


In [31]:
for col in combo_df.columns:
    print(col)

Date
Unnamed: 0_x
death_id
death_date
age
gender
city
case_status
count
Unnamed: 0_y
CaseCount
ReportedDate
Sex
Age
Type
City
admission_count
kpi_admits_indicator
inpatient_count
kpi_patient_indicator
inpatient_count_pct_change
Unnamed: 0
daily number of doses received by Larimer County residents
total number of doses recevied by residents
daily number of residents receiving first dose
total number of residents receiving first dose
daily number of residents vaccinated
total number of residents vaccinated
daily number of 70+ vaccinated
total number of 70+ vaccinated
daily number of 70+ at least one dose
total number of 70+ at least one dose
daily number of Latinx residents vaccinated
total of Latinx residents vaccinated
daily number of White non-Latinx residents vaccinated
total of White non-Latinx residents vaccinated
daily number of non-White non-Latinx residents vaccinated
total of non-White non-Latinx residents vaccinated
dailyUnknown
totalUnknown
daily_additional_doses
total_additi

In [32]:
combo_df.rename(columns = {'count':'Daily Death Count',
                           'daily number of doses received by Larimer County residents':'Daily doses',
                           'CaseCount':'Daily Cases',
                           'admission_count':'Daily Hospitalizations'
                          }, inplace = True)

In [33]:
combo_df[['Date','Daily doses','Daily Cases','Daily Hospitalizations','Daily Death Count']]

,Date,Daily doses,Daily Cases,Daily Hospitalizations,Daily Death Count
0,2020-03-09,NaN,1,<NA>,1.0
1,2020-03-10,NaN,0,<NA>,0.0
2,2020-03-11,NaN,0,<NA>,0.0
3,2020-03-12,NaN,0,<NA>,0.0
4,2020-03-13,NaN,0,<NA>,1.0
...,...,...,...,...,...
683,2022-01-21,1149.0,593,0,NaN
684,2022-01-22,777.0,494,0,NaN
685,2022-01-23,172.0,450,0,NaN
686,2022-01-24,251.0,444,11,NaN


In [34]:
print(combo_df.isna().sum().sum())

6083


In [35]:
combo_df = combo_df.fillna(0)
print(combo_df.isna().sum().sum())

0


In [36]:
combo_df[['Date','Daily doses','Daily Cases','Daily Hospitalizations','Daily Death Count']]

,Date,Daily doses,Daily Cases,Daily Hospitalizations,Daily Death Count
0,2020-03-09,0.0,1,0,1.0
1,2020-03-10,0.0,0,0,0.0
2,2020-03-11,0.0,0,0,0.0
3,2020-03-12,0.0,0,0,0.0
4,2020-03-13,0.0,0,0,1.0
...,...,...,...,...,...
683,2022-01-21,1149.0,593,0,0.0
684,2022-01-22,777.0,494,0,0.0
685,2022-01-23,172.0,450,0,0.0
686,2022-01-24,251.0,444,11,0.0


# BOOKMARK


- [ ] Get overall plot layout
- [ ] Make plots
- [x] Figure out hovertips

```python
show(row(column(fig1, fig2), column(fig3)))
```


## Visualize

In [37]:
reset_output()
output_notebook()

Loading BokehJS ...

```python
hover = HoverTool()
hover.tooltips = [
    ("Totals", "@TONS_HE High Explosive / @TONS_IC Incendiary / @TONS_FRAG Fragmentation")]

hover.mode = 'vline'

p.add_tools(hover)

show(p)
```

In [38]:
lar_vac_data = ColumnDataSource(combo_df)


TOOLS = 'pan, wheel_zoom, box_zoom, reset, save'

TOOLTIPS = [('Date', '@Date{%F}'), 
            ('Daily Doses','@{Daily doses}')]


daily_vac_figure_bar = figure(title='Daily Vaccinations',
                              x_axis_type="datetime",
                              tools=TOOLS)

daily_vac_figure_bar.vbar(x='Date',
                          top='Daily doses',
                          width=datetime.timedelta(days=0.5),
                          source=lar_vac_data)

hover = HoverTool()

hover.tooltips = TOOLTIPS

hover.mode = 'vline'

hover.formatters = {'@Date' : 'datetime'}

daily_vac_figure_bar.add_tools(hover)

#daily_vac_figure_bar.add_tools(HoverTool(tooltips=[("Date", "@x"),('Doses','@top')]))
                   

show(daily_vac_figure_bar)

In [39]:
lar_vac_data = ColumnDataSource(larimer_vac)

reset_output()
output_notebook()

TOOLS = 'pan, wheel_zoom, box_zoom, reset, hover, save'

# x = lar_vac_data['Date']
# y = lar_vac_data('daily number of doses received by Larimer County residents')

TOOLTIPS = [('Date', '@Date{%F}'), 
            ('Daily Doses','@{daily number of doses received by Larimer County residents}')]

daily_vac_figure = figure(title='Daily Vaccinations',
                         x_axis_type="datetime",
                         tools=TOOLS,
                         tooltips=TOOLTIPS)

daily_vac_figure.line(x='Date',
                      y='daily number of doses received by Larimer County residents',
                      source=lar_vac_data)


hover = daily_vac_figure.select_one(HoverTool)


hover.formatters = {'@Date' : 'datetime'}

show(daily_vac_figure)

Loading BokehJS ...

In [49]:
combo_data = ColumnDataSource(data = combo_df)


TOOLS = 'pan, wheel_zoom, box_zoom, reset, save'

TOOLTIPS = [('Date', '@Date{%F}'), 
            ('Daily Cases','@{Daily Cases}')]

combo_data_figure = figure(title='Daily Cases',
                           x_axis_type="datetime",
                           tools=TOOLS)

combo_data_figure.vbar(x='Date',
                       top='Daily Cases',
                       width=datetime.timedelta(days=0.5),
                       source=combo_data)


hover = HoverTool()

hover.tooltips = TOOLTIPS

hover.mode = 'vline'

hover.formatters = {'@Date' : 'datetime'}

combo_data_figure.add_tools(hover)

#hover.formatters = {'@Date' : 'datetime'}



show(combo_data_figure)

In [41]:
combo_data = ColumnDataSource(combo_df)

reset_output()
output_notebook()


combo_data_figure = figure(title='Daily Hospitalizations',
                           x_axis_type="datetime")

combo_data_figure.vbar(x='Date',
                       top='Daily Hospitalizations',
                       source=combo_data)



show(combo_data_figure)

Loading BokehJS ...

In [42]:
combo_data = ColumnDataSource(combo_df)

reset_output()
output_notebook()


combo_data_figure = figure(title='Daily Deaths',
                           x_axis_type="datetime")

combo_data_figure.vbar(x='Date',
                       top='Daily Death Count',
                       source=combo_data)



show(combo_data_figure)

Loading BokehJS ...